# Gemini 3 Script Generator with Knowledge Graph

This notebook demonstrates how to use the Gemini 3 API (via `google-genai`) to generate scripts based on user input, enriched with context from a Neo4j Knowledge Graph.

## Prerequisites
- Neo4j Database running (with the Fabric Knowledge Graph loaded)
- Google Gemini API Key


In [ ]:
# Install dependencies
%pip install -q -r requirements.txt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import json
import numpy as np
from dotenv import load_dotenv, find_dotenv
from neo4j import GraphDatabase, basic_auth
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from google import genai
from google.genai import types

# Load environment variables
# Try to find .env file explicitly if not found automatically
dotenv_path = find_dotenv()
if dotenv_path:
    print(f"Loading .env from: {dotenv_path}")
    load_dotenv(dotenv_path, override=True)
else:
    print("⚠️ .env file not found. Please ensure it exists in the project root.")
    # Fallback: try loading from current directory
    load_dotenv(override=True)

print("Libraries imported successfully.")

Loading .env from: /Users/shreyasjagannath/dev/fabric/onfabric-data-science-interview/.env
Libraries imported successfully.


In [3]:
# Neo4j Configuration
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = None
        try:
            self.driver = GraphDatabase.driver(uri, auth=basic_auth(user, password))
            with self.driver.session() as session:
                session.run("RETURN 1")
                print("✓ Successfully connected to Neo4j")
        except Exception as e:
            print(f"✗ Connection failed: {e}")
            
        
    def close(self):
        if self.driver:
            self.driver.close()
    
    def query(self, query_str, **kwargs):
        with self.driver.session() as session:
            return session.run(query_str, **kwargs).data()

# Initialize connection
neo4j_conn = Neo4jConnection(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)

✓ Successfully connected to Neo4j


In [4]:
# Initialize Gemini Client
# Ensure you have GOOGLE_API_KEY or GEMINI_API_KEY in your .env file
api_key = os.getenv("GOOGLE_API_KEY") or os.getenv("GEMINI_API_KEY")

if not api_key:
    print("⚠️ GOOGLE_API_KEY or GEMINI_API_KEY not found in environment variables.")
    print("Gemini features will be disabled until a valid key is provided.")
    client = None
else:
    client = genai.Client(api_key=api_key)
    print("Gemini client initialized.")

Gemini client initialized.


In [5]:
# Initialize Embedding Model
print("Loading embedding model (this may take a moment)...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✓ Embedding model loaded.")

def get_embedding(text):
    """Generate embedding for text using sentence transformers."""
    if not text or not isinstance(text, str):
        return None
    try:
        clean_text = text.strip()[:512]
        embedding = embedding_model.encode(clean_text)
        # Convert to list of Python floats (not numpy types)
        return [float(x) for x in embedding.tolist()]
    except Exception as e:
        print(f"Error embedding text: {e}")
        return None

def search_knowledge_graph(query_text, node_types=['Topic', 'Entity'], limit=10):
    """
    Search the knowledge graph using Neo4j vector indexes for semantic similarity.
    
    Args:
        query_text: Natural language query
        node_types: List of node types to search (['Topic', 'Entity', 'Category'])
        limit: Number of results to return per node type
    
    Returns:
        List of relevant items with type, content, and similarity score
    """
    if not neo4j_conn:
        print("⚠️ Neo4j connection not available")
        return []
    
    query_embedding = get_embedding(query_text)
    if query_embedding is None:
        print("⚠️ Could not generate embedding for query")
        return []
    
    all_results = []
    
    # Map node types to their vector indexes
    index_map = {
        'Entity': 'entity_embedding_idx',
        'Topic': 'topic_embedding_idx',
        'Category': 'category_embedding_idx'
    }
    
    # Query each node type using its vector index
    for node_type in node_types:
        if node_type not in index_map:
            continue
            
        index_name = index_map[node_type]
        
        # Use Neo4j vector index for efficient semantic search
        vector_query = f"""
        CALL db.index.vector.queryNodes('{index_name}', $k, $embedding)
        YIELD node, score
        RETURN 
            labels(node)[0] as node_type,
            coalesce(node.name, node.content) as content,
            score as similarity
        ORDER BY similarity DESC
        LIMIT $limit
        """
        
        try:
            results = neo4j_conn.query(
                vector_query,
                embedding=query_embedding,
                k=limit * 2,  # Query more, filter to limit
                limit=limit
            )
            all_results.extend(results)
        except Exception as e:
            print(f"⚠️ Error querying {node_type}: {e}")
    
    # Sort all results by similarity and return top results
    all_results = sorted(all_results, key=lambda x: x['similarity'], reverse=True)
    
    # Format results for consistency
    formatted_results = []
    for result in all_results[:limit]:
        formatted_results.append({
            'type': result['node_type'],
            'content': result['content'],
            'similarity': float(result['similarity'])
        })
    
    return formatted_results

Loading embedding model (this may take a moment)...
✓ Embedding model loaded.
✓ Embedding model loaded.


In [10]:
def generate_script_with_context(user_request):
    """
    Generates a video/movie script based on user request, using Knowledge Graph context.
    Uses Neo4j vector indexes for efficient semantic search.
    """
    print(f"🔍 Analyzing request: '{user_request}'")
    
    # 1. Get Context from Knowledge Graph using vector search
    print("   Querying Knowledge Graph with vector indexes...")
    
    # Search both Topics and Entities for comprehensive context
    kg_results = search_knowledge_graph(
        user_request, 
        node_types=['Topic', 'Entity', 'Category'],
        limit=15
    )
    
    context_str = ""
    if kg_results:
        print(f"\n   --- Found {len(kg_results)} Relevant Items ---")
        
        # Group results by type for better context organization
        topics = [r for r in kg_results if r['type'] == 'Topic']
        entities = [r for r in kg_results if r['type'] == 'Entity']
        categories = [r for r in kg_results if r['type'] == 'Category']
        
        context_str = "Relevant Context from Knowledge Graph (User's Interests & Activity):\n\n"
        
        if categories:
            context_str += "User Interest Categories:\n"
            for cat in categories[:3]:
                context_str += f"  - {cat['content']} (relevance: {cat['similarity']:.2f})\n"
                print(f"   [Category] {cat['content']} (similarity: {cat['similarity']:.3f})")
            context_str += "\n"
        
        if topics:
            context_str += "Related Topics from User Activity:\n"
            for topic in topics[:5]:
                context_str += f"  - {topic['content']} (relevance: {topic['similarity']:.2f})\n"
                print(f"   [Topic] {topic['content'][:60]}... (similarity: {topic['similarity']:.3f})")
            context_str += "\n"
        
        if entities:
            context_str += "Specific Items/Brands User Has Engaged With:\n"
            for entity in entities[:7]:
                context_str += f"  - {entity['content']} (relevance: {entity['similarity']:.2f})\n"
                print(f"   [Entity] {entity['content']} (similarity: {entity['similarity']:.3f})")
        
        print("   " + "-"*50)
        
    else:
        context_str = "No specific context found in Knowledge Graph."
        print("   ⚠️ No relevant context found")
        
    print(f"\n   ✓ Retrieved {len(kg_results)} relevant items from knowledge graph")
    
    # 2. Construct Enhanced Prompt for Gemini
    prompt = f"""
    You are an expert creative writer and video scriptwriter.
    
    User Request: {user_request}
    
    {context_str}
    
    Task: Create a creative video/movie script that addresses the user's request, personalized to their interests.
    
    CRITICAL CONSTRAINT: The video must be exactly 8 seconds long. The storyline must be concise, impactful, and complete within this short duration.
    
    Guidelines:
    1. **Personalization**: Use the knowledge graph context above to tailor the script to the user's specific interests, 
       activities, and preferences. Reference relevant topics, categories, and items they've engaged with.
    2. **Structure**: Format the script with:
       - **Title**: Catchy and relevant title
       - **Logline**: A one-sentence summary of the short storyline.
       - **Scene/Setting Description**: Clear visual setting.
       - **Action/Visuals**: Detailed visual cues for the 8-second sequence.
       - **Audio/Voiceover**: Minimal dialogue or voiceover, strictly timed (0:00-0:08).
    3. **Storyline**: Create a micro-narrative with a clear beginning, middle, and end, even within 8 seconds.
    4. **Tone**: Match the tone to the user's request and their interest profile.
    5. **Length**: Keep the script concise. Focus on visual storytelling.
    
    Output the script in clear Markdown format.
    """
    
    # 3. Call Gemini API
    print("   Generating personalized script with Gemini...")
    
    if client is None:
        return "❌ Gemini Client not initialized. Please provide GOOGLE_API_KEY."

    try:
        response = client.models.generate_content(
            model="gemini-3-pro-preview",
            contents=prompt
        )
        print("   ✓ Script generated successfully\n")
        return response.text
    except Exception as e:
        return f"❌ Error generating content: {e}"

print("✓ Generator function defined with vector search integration.")

✓ Generator function defined with vector search integration.


In [11]:
import time
import requests
import os
import datetime

def create_video_from_script(script_text):
    """
    Extracts a visual prompt from the script and generates a video using Veo 3.
    """
    if not script_text:
        print("❌ No script provided.")
        return


    # 1. Generate Video with Veo 3
    print(f"\n🎥 Generating video with Veo...")
    
    # Initialize v1alpha client for Veo
    try:
        from google.genai.types import HttpOptions
        alpha_client = genai.Client(
            api_key=api_key,
            http_options=HttpOptions(api_version='v1alpha')
        )
        print("   Initialized v1alpha client for Veo.")
    except Exception as e:
        print(f"   ⚠️ Could not initialize v1alpha client ({e}), using default.")
        alpha_client = client

    # model_name = "veo-3.1-generate-preview"
    model_name = "veo-3.1-fast-generate-preview"
    
    print(f"   Attempting with model: {model_name}...")

    try:
        # Start the video generation operation
        operation = alpha_client.models.generate_videos(
            model=model_name,
            prompt=script_text,
        )
        
        print("   Operation started. Waiting for video generation to complete...")
        
        # Poll the operation status until the video is ready.
        while not operation.done:
            print("   ...still processing...")
            time.sleep(10)
            operation = alpha_client.operations.get(operation)
            
        if operation.result:
             # Download the generated video.
            generated_video = operation.result.generated_videos[0]
            
            # Create unique filename with timestamp in videos folder
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            output_dir = "videos"
            os.makedirs(output_dir, exist_ok=True)
            output_file = os.path.join(output_dir, f"generated_video_{timestamp}.mp4")
            
            # Check if we have a URI to download from
            video_uri = getattr(generated_video.video, 'uri', None)
            
            if video_uri:
                # Append API key for authentication
                authenticated_uri = f"{video_uri}&key={api_key}"
                print(f"   Downloading video from authenticated URI...")
                
                try:
                    response = requests.get(authenticated_uri, stream=True)
                    response.raise_for_status()
                    
                    with open(output_file, "wb") as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            if chunk:
                                f.write(chunk)
                    print(f"✓ Video saved successfully to {output_file}")
                except Exception as download_err:
                    print(f"   ❌ Failed to download video from URI: {download_err}")
            else:
                print("   ⚠️ No URI found in generated video object. Trying fallback download...")
                # Fallback: try using the client's file download if URI is missing but name exists
                try:
                    video_content = alpha_client.files.content(file_name=generated_video.video.name)
                    with open(output_file, "wb") as f:
                        f.write(video_content)
                    print(f"✓ Video saved successfully to {output_file}")
                except Exception as e:
                     print(f"   ❌ Fallback download failed: {e}")
                     print(f"   Debug info - generated_video.video: {generated_video.video}")

        else:
            print("   ❌ Operation completed but no result returned.")

    except Exception as e:
        print(f"   ❌ Failed with {model_name}: {e}")


In [12]:
# Example Usage
user_request = "travel to paradise"
script = generate_script_with_context(user_request)

print("\n" + "="*40 + "\nGENERATED SCRIPT\n" + "="*40 + "\n")
print(script)

🔍 Analyzing request: 'travel to paradise'
   Querying Knowledge Graph with vector indexes...

   --- Found 15 Relevant Items ---
   [Topic] To Paradise review... (similarity: 0.879)
   [Topic] shopping paradise... (similarity: 0.842)
   [Topic] Travel destination... (similarity: 0.775)
   [Topic] Travel destination... (similarity: 0.775)
   [Topic] Travel destination... (similarity: 0.775)
   [Entity] To Paradise (similarity: 0.936)
   [Entity] Dayal Paradise (similarity: 0.837)
   [Entity] Paradise Road (similarity: 0.831)
   [Entity] travel (similarity: 0.787)
   --------------------------------------------------

   ✓ Retrieved 15 relevant items from knowledge graph
   Generating personalized script with Gemini...
   ✓ Script generated successfully


GENERATED SCRIPT

Here is a personalized creative video script tailored to your specific interests in literature, luxury shopping, and travel destinations.

**Title:** Pages to Places
**Logline:** A seamless transition from reading the 

In [13]:

# Run the video generation
create_video_from_script(script)


🎥 Generating video with Veo...
   Initialized v1alpha client for Veo.
   Attempting with model: veo-3.1-fast-generate-preview...
   Operation started. Waiting for video generation to complete...
   ...still processing...
   ...still processing...
   ...still processing...
   ...still processing...
   ...still processing...
   ...still processing...
✓ Video saved successfully to videos/generated_video_20251214_184943.mp4


### List of Video Generation Models
- models/veo-2.0-generate-001
- models/veo-3.0-generate-001
- models/veo-3.0-fast-generate-001
- models/veo-3.1-generate-preview
- models/veo-3.1-fast-generate-preview

### List of Gemini Text Models

 - models/gemini-2.5-flash
 - models/gemini-2.5-pro
 - models/gemini-2.0-flash-exp
 - models/gemini-2.0-flash
 - models/gemini-2.0-flash-001
 - models/gemini-2.0-flash-lite-001
 - models/gemini-2.0-flash-lite
 - models/gemini-2.0-flash-lite-preview-02-05
 - models/gemini-2.0-flash-lite-preview
 - models/gemini-exp-1206
 - models/gemini-2.5-flash-preview-tts
 - models/gemini-2.5-pro-preview-tts
 - models/gemma-3-1b-it
 - models/gemma-3-4b-it
 - models/gemma-3-12b-it
 - models/gemma-3-27b-it
 - models/gemma-3n-e4b-it
 - models/gemma-3n-e2b-it
 - models/gemini-flash-latest
 - models/gemini-flash-lite-latest
 - models/gemini-pro-latest
 - models/gemini-2.5-flash-lite
 - models/gemini-2.5-flash-image-preview
 - models/gemini-2.5-flash-image
 - models/gemini-2.5-flash-preview-09-2025
 - models/gemini-2.5-flash-lite-preview-09-2025
 - models/gemini-3-pro-preview